In [39]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("groq_api_key")
print(groq_api_key)

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


In [40]:
## using model from groq 
from langchain_groq import ChatGroq
model = ChatGroq(model = "gemma2-9b-it", groq_api_key = groq_api_key)

In [41]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027975345CD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002795F405610>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [42]:
from langchain_core.messages import HumanMessage
model.invoke(
    [
        HumanMessage(content = "Hi I am pratyush I work as data scientist")
    ]
)

AIMessage(content="Hi Pratyush! It's nice to meet you.  \n\nThat's awesome that you're a data scientist! What kind of projects do you work on?  \n\nI'm always eager to learn more about the work people do in this field.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 20, 'total_tokens': 78, 'completion_time': 0.105454545, 'prompt_time': 8.1369e-05, 'queue_time': 0.018314557999999998, 'total_time': 0.105535914}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-609b19a5-6305-405b-9b01-fd920a6cf261-0', usage_metadata={'input_tokens': 20, 'output_tokens': 58, 'total_tokens': 78})

In [43]:
## now copying ad pasting this ai message to check if it remembers the history 
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hi I am pratyush I work as data scientist"),
        AIMessage(content = "Hi Pratyush!  \n\nThat's great! Data science is a fascinating field.  What kind of work do you do as a data scientist?  Are you working on any particularly interesting projects right now? \n\nI'm always eager to learn more about how people are using data to make a difference.t"),
         HumanMessage(content = "WHAT DOes pratyush do?")
    ]
)

## able to remember contect when messages are passed in the same list 

AIMessage(content="You're asking me about Pratyush, who you mentioned is a data scientist.  \n\nSince I don't have access to personal information about individuals like Pratyush, I can only tell you what data scientists generally do:\n\n* **Analyze data:** Data scientists collect, clean, and analyze large datasets to find patterns and insights.\n* **Build models:** They use statistical and machine learning techniques to build predictive models that can forecast future trends or outcomes.\n* **Communicate findings:** Data scientists present their findings to stakeholders in a clear and concise way, using visualizations and storytelling.\n* **Solve problems:** They use data-driven insights to help organizations solve problems in areas such as marketing, finance, healthcare, and more.\n\nTo learn more about what Pratyush specifically does, you'd have to ask him directly! 😊  \n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 178, 'prompt_tokens': 104, 

In [44]:
## some people in a team would be using chatgpt ,others might be using groq and so on 
## when such people talk to each other how does conext remembered

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [45]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### creating a method that stores messages with each session
store = {}
def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## if session is not there then ChatMessageHistory initialized for that particular seesion 
## otherwise it is retrieved

In [46]:
config = {"configurable": {"session_id":"chat1"}}

In [47]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)
## takes input as model and function based on which it would attach model with all the history that it has seen in a particular session

In [48]:
response = with_message_history.invoke(
    [HumanMessage(content= "hi my name is pratyush i am datascientist")], config = config
)

##using in context of session where session_id = chat1

In [49]:
response.content

"Hi Pratyush, it's nice to meet you!\n\nBeing a fellow AI, I'm always interested in connecting with people in the data science field. What kind of work do you specialize in as a data scientist? \n\nDo you have any projects you're particularly excited about or challenges you're currently facing that you'd like to discuss?\n"

In [50]:
response = with_message_history.invoke(
    [HumanMessage(content= "what is my name")], config = config
)

# in session id chat1 which is being stored ealier also 

In [51]:
response.content

'Your name is Pratyush.  You told me at the beginning of our conversation! 😊  \n\nIs there anything else I can help you with?\n'

In [52]:
## change_config
cconfig1 = {"configurable": {"session_id": "chat2"}}

In [53]:
response = with_message_history.invoke(
    [HumanMessage("WHAT IS MY NAME")], config = cconfig1
)

In [54]:
response.content

"As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","behave like an ai chatbot and return results that are concise and as per your best judgement"),
        MessagesPlaceholder(variable_name= "message")
    ]   
    
)

In [56]:
chain = prompt|model

In [57]:
chain.invoke({"message" : [HumanMessage(content = "hello my name is pratyush")]})

AIMessage(content='Hello, Pratyush! \n\nWhat can I do for you today? 😊  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 34, 'total_tokens': 56, 'completion_time': 0.04, 'prompt_time': 0.000374859, 'queue_time': 0.0140383, 'total_time': 0.040374859}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c5cc4046-ab8e-4c19-8918-78a65314e8e0-0', usage_metadata={'input_tokens': 34, 'output_tokens': 22, 'total_tokens': 56})

In [58]:
### using with message history 
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [59]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
   { "message": [HumanMessage(content = "Hello my name is pratyush")] },config = config
)

In [60]:
response.content

'Hello Pratyush, nice to meet you! What can I do for you?  \n\n'

In [61]:
prompt = ChatPromptTemplate.from_messages(
[
    ("system","you are an intelligent chatbot , give answers as per your best knowledge in {language}"),
    MessagesPlaceholder("messages")
]
)
chain = prompt|model

In [62]:
##invoking response with 2 keys
reponse = chain.invoke({"messages" : [HumanMessage("Hi My name is pratyush")], "language" :  "hindi"})

In [63]:
reponse.content

'नमस्ते प्रद्युष! \n\nमुझे आपसे मिलकर बहुत खुशी हो रही है। मैं एक बुद्धिमान चैटबॉट हूँ और मैं अपनी पूरी कोशिश करूँगा कि आपकी सभी प्रश्नों के उत्तर देने में आपकी सहायता कर सकूँ।  क्या मैं आपको कुछ मदद कर सकता हूँ? \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [64]:
with_message_history  = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages"
)

In [65]:
config1 = {"configurable" :{"session_id":"chat4"}}

response = with_message_history.invoke(
    {
       "messages" : [HumanMessage(content = "HI i am pratyush")], "language" :"french"
    },config = config
)

In [66]:
response.content

"Bonjour Pratyush, ravi de te rencontrer ! Comment vas-tu aujourd'hui ? \n\n\n\n"

In [67]:
response = with_message_history.invoke(
   { "messages" : [HumanMessage(content = "what is my name")], "language" : "french"}, config = config
)

In [68]:
response.content

'Votre nom est Pratyush.  😄  \n\n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [69]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last",
    ## starts counting tokens from last message
    token_counter = model,
    ## model is used to count tokens
    include_system = True,
    allow_partial = False,
    ## remove the partial messages
    start_on = "human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [70]:
prompt = ChatPromptTemplate.from_messages(
[
    ("system","you are an intelligent chatbot , give answers as per your best knowledge in {language}"),
    MessagesPlaceholder("messages")
]
)
chain = prompt|model

In [71]:
## chains along ith this

from operator import itemgetter 
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages1")|trimmer)
    |prompt
    | model
)
## prompt above with message and language 2 params used
response = chain.invoke(
    {
        "messages1":messages + [HumanMessage(content="What ice cream do i like")],
        "language" : "Hindi" 
    }
)
response.content


'मुझे तो तुम्हारी पसंदीदा आइसक्रीम के बारे में पता नहीं है!  \n\nतुम्हें किस स्वाद में आइसक्रीम सबसे ज़्यादा पसंद है? 🍦😋  \n'

In [72]:
response = chain.invoke(
    {
        "messages1": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2".  😊 \n'

In [73]:
## wrapping this in a messsage history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages1",
    
)

config = {"configurable" : {"session_id":"chat5"}}

In [74]:
response = with_message_history.invoke(
    {
        "messages1": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations and don't know your name.  \n\nWould you like to tell me your name? 😊  \n"

In [ ]:
response = with_message_history.invoke(
    {
        "messages1": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help!  \n\nWhat's on your mind? 😊  \n\n"

: 